In [1]:
import votekit.ballot_generator.std_generator.spacial as spacial
import votekit.ballot_generator as bg
import numpy as np

class VoterType:
    
    def __init__(self, generator,name=""):
        self.gen = generator
        self.name = name

    def __call__(self):
        # Generate one voter
        return self.gen()

    def sample(self, n):
        # Generate n voters
        return np.array([self.gen() for _ in range(n)])

    def uniform_L_infinity_ball(x,y,r=1,name=""):
        # Uniform Distribution
        gen = lambda: np.random.uniform(low= (x-r,y-r),high= (x+r,y+r),size= 2)
        return VoterType(gen,name)

    def uniform_L1_ball(x,y,r=1, name=""):
        # L1 Distribution
        def gen():
            numbers = np.random.exponential(1, size = 2) # Picks two random numbers 
            normalized_numbers = numbers / numbers.sum() # Normalize so their sum adds to one
            signs = np.random.choice([-1,1], size = 2])
            points = signs * normalized_numbers # Some point on the boundary
            radius = r * (np.random.uniform(0,1) ** (1/2))
            return np.array([x,y]) + radius * points
        return VoterType(gen,name)

    def uniform_L2_ball(x,y,r=1,name=""):
        #L2 Distribution
        def gen():
            # Pick angle and radius on a circle of radius 1
            theta = np.random.uniform(0, 2*np.pi)
            radius = r * np.sqrt(np.random.uniform(0, 1))
            return np.array([x, y]) + rad * np.array([np.cos(theta), np.sin(theta)])
        return VoterType(gen, name)

    def gaussian(x, y, sigma=0.5, name=""):
        # Gaussian Moment, sigma is the standard deviation 
        gen = lambda: np.random.normal(loc=(x, y), scale=sigma, size=2)
        return VoterType(gen, name)

    def correlated_gaussian(mean, cov, name=""):
        #Correlated Gaussian with full covariance.
        mean = np.array(mean, dtype=float)
        cov = np.array(cov, dtype=float)
        gen = lambda: np.random.multivariate_normal(mean= mean, cov=cov)
        return VoterType(gen, name)

    

class Region:
    def __init__(self,voter_types,amounts,name=""):
        self.voter_types=voter_types
        self.amounts = amounts
        assert len(self.voter_types) == len(self.amounts), "Dude What"
        
        tot = sum(amounts)
        self.proportions = [x/tot for x in amounts]

    def gen_one_random(self):
        my_guy = np.random.choice(self.voter_types, p=self.proportions)
        return my_guy.gen()

def voter_generator_R_sep(R=0):
    first_rand = {"low": (-1,-1),"high": (1,1), "size": 2}
    second_rand = {"low": (-1,R-1),"high": (1,R+1), "size": 2}
    choose = np.random.rand(1)
    if choose < 1/2:
        return np.random.uniform(**first_rand)
    else:
        return np.random.uniform(**second_rand)

def candidate_generator_R_sep(R=0):
    return np.random.uniform(**{"low": (-1,-1), "high": (-1,R+1), "size": 2})
    

In [2]:
voter_type_K = VoterType.uniform_centered_L1(0,0,name="K")
voter_type_L = VoterType.uniform_centered_L1(2,2,name="K")
region_1 = Region([voter_type_K,voter_type_L],[5,20])

bg.spacial_profile_and_positions_generator(
    number_of_ballots= 30,
    candidates= ["A","B"],
    voter_dist=region_1.gen_one_random,
    candidate_dist = np.random.uniform,
    candidate_dist_kwargs={"low": (-1,-1), "high": (-1,+1), "size": 2})

(RankProfile
 Maximum ranking length: 2
 Candidates: ('A', 'B')
 Candidates who received votes: ('B', 'A')
 Total number of Ballot objects: 30
 Total weight of Ballot objects: 30,
 {'A': array([-1.        ,  0.15956494]),
  'B': array([-1.       , -0.4367838])},
 array([[ 2.82969922,  1.48840969],
        [ 1.92025359,  2.15409286],
        [ 2.31905636,  2.06137148],
        [ 1.40131031,  1.71976368],
        [ 2.90795226,  2.7407825 ],
        [ 2.61093819,  2.83257271],
        [ 2.31244266,  2.29613245],
        [ 2.86225266,  1.18250978],
        [ 1.13684833,  2.60471457],
        [ 1.54269192,  1.38539401],
        [ 1.50437026,  1.28285274],
        [-0.65365595, -0.41100432],
        [ 2.45521713,  1.42296363],
        [ 1.21768668,  1.1706659 ],
        [ 1.1208352 ,  1.69368329],
        [ 2.09517195,  1.99849803],
        [ 1.90527651,  2.70019886],
        [-0.1351928 , -0.60101165],
        [ 0.50358725, -0.6935175 ],
        [-0.05854895, -0.38816006],
        [ 0.61655